### Домашнее задание к теме:   
**Гибридные рекомендательные системы**

Задача - сделать гибридную рекомендательную систему

Использовать датасет [MovieLens](https://grouplens.org/datasets/movielens/latest/)  

Система должна выводить рекомендации фильмов для пользователя на основе рейтингов, которые он поставил.

Гибридность означает комбинированный подход к решению задачи. Выберем топ-фильмов на основе всех оценок всех пользователей и согласно этого списка порекомендуем конкретному пользователю фильмы.

### Найдём список топ фильмов

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
#links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
#tags = pd.read_csv('tags.csv')

In [3]:
movies.shape, ratings.shape

((9742, 3), (100836, 4))

In [12]:
movies_with_ratings = pd.merge(ratings, movies, on='movieId')

In [13]:
movies_with_ratings.shape

(100836, 6)

In [14]:
movies_with_ratings.sample(5)

,userId,movieId,rating,timestamp,title,genres
71792,591,2908,2.0,970524787,Boys Don't Cry (1999),Drama
73816,380,61248,3.0,1494696360,Death Race (2008),Action|Adventure|Sci-Fi|Thriller
12218,452,2406,4.0,1013395071,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
75471,422,4220,4.0,986172320,"Longest Yard, The (1974)",Comedy
35578,219,4643,2.5,1194932356,Planet of the Apes (2001),Action|Adventure|Drama|Sci-Fi


In [15]:
movies_with_ratings.isnull().any().any()

False

**Нет нулевых значений. Можно продолжать.**

### Топ-фильмы
Предложим в качестве **метрики**, помогающий выбрать **топ-фильмы** количество отзывов, умноженное на среднюю оценку.

$\odot$ количество рейтингов для фильмов

In [16]:
title_number_ratings = {}

for title,group in tqdm_notebook(movies_with_ratings.groupby('title')):
    title_number_ratings[title] = len(group.userId.unique())

$\odot$ у каких фильмов больше отзывов

In [17]:
sorted(title_number_ratings.items(), key=lambda x: x[1], reverse=True)[:10]

[('Forrest Gump (1994)', 329),
 ('Shawshank Redemption, The (1994)', 317),
 ('Pulp Fiction (1994)', 307),
 ('Silence of the Lambs, The (1991)', 279),
 ('Matrix, The (1999)', 278),
 ('Star Wars: Episode IV - A New Hope (1977)', 251),
 ('Jurassic Park (1993)', 238),
 ('Braveheart (1995)', 237),
 ('Terminator 2: Judgment Day (1991)', 224),
 ("Schindler's List (1993)", 220)]

$\odot$ средние оценки для фильмов

In [18]:
title_mean_rating = movies_with_ratings.groupby('title')['rating'].mean()

топ-10 средних оценок

In [19]:
title_mean_rating.sort_values(ascending=False)[:10]

title
Karlson Returns (1970)                           5.0
Winter in Prostokvashino (1984)                  5.0
My Love (2006)                                   5.0
Sorority House Massacre II (1990)                5.0
Winnie the Pooh and the Day of Concern (1972)    5.0
Sorority House Massacre (1986)                   5.0
Bill Hicks: Revelations (1993)                   5.0
My Man Godfrey (1957)                            5.0
Hellbenders (2012)                               5.0
In the blue sea, in the white foam. (1984)       5.0
Name: rating, dtype: float64

---

**Перейдём к вычислению метрики.**

Дополнительные статистические параметры:

In [21]:
title_number_ratings_values = list(title_number_ratings.values())

min_number_ratings = np.min(title_number_ratings_values)
max_number_ratings = np.max(title_number_ratings_values)
mean_number_ratings = np.mean(title_number_ratings_values)

min_number_ratings.round(4), max_number_ratings.round(4), mean_number_ratings.round(4)

(1, 329, 10.3747)

$\odot$ метрика для каждого фильма

In [22]:
films_marks = {}
for title in title_number_ratings.keys():
    films_marks[title] = title_mean_rating[title] * \
    (title_number_ratings[title] - mean_number_ratings) / (max_number_ratings - min_number_ratings)

---

**Список топ-фильмов:**

In [23]:
#зададимся числом лучших фильмов
top_films_number = 1000

In [24]:
top_films = sorted(films_marks.items(), key = lambda x: x[1], reverse=True)[:top_films_number]
print('Лучшие 15 фильмов из топ-1000:')
top_films[:15]

Лучшие 15 фильмов из топ-1000:


[('Shawshank Redemption, The (1994)', 4.140396622352077),
 ('Forrest Gump (1994)', 4.04511657667948),
 ('Pulp Fiction (1994)', 3.795599234431761),
 ('Matrix, The (1999)', 3.4207454409691413),
 ('Silence of the Lambs, The (1991)', 3.4080113927564395),
 ('Star Wars: Episode IV - A New Hope (1977)', 3.103974793934813),
 ('Braveheart (1995)', 2.7855877015865484),
 ('Fight Club (1999)', 2.7047848943888955),
 ("Schindler's List (1993)", 2.7002035552689096),
 ('Jurassic Park (1993)', 2.6024230574258618),
 ('Terminator 2: Judgment Day (1991)', 2.5862869902088406),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 2.5785484011187134),
 ('Usual Suspects, The (1995)', 2.5016296926169597),
 ('Toy Story (1995)', 2.4461018531687673),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  2.4324644021391335)]

### Рекомендация для конкретного пользователя

Пусть гипотетический пользователь посмотрел следующие фильмы:

In [27]:
new_user_films = movies_with_ratings.title.sample(7)

In [28]:
new_user_films

11994    I Still Know What You Did Last Summer (1998)
10316                      Wedding Singer, The (1998)
49830                             Donnie Darko (2001)
70257           Fear and Loathing in Las Vegas (1998)
84239                        King of Kong, The (2007)
22779                             Little Voice (1998)
14255                    From Russia with Love (1963)
Name: title, dtype: object

In [36]:
nuf_idx = new_user_films.index

In [38]:
new_user_films[nuf_idx[0]]

'I Still Know What You Did Last Summer (1998)'

и оценил их (создадим словарик конкретного пользователя).  
    Это будет новый пользователь, отсутсвующий в нашей базе (или как вариант, возможно, на какого-то пользователя нашла амнезия и он решил пересмотреть серию про агента Борна). )

In [41]:
new_user_ratings = {
    new_user_films[nuf_idx[0]]: 5.0,
    new_user_films[nuf_idx[1]]: 4.5,
    new_user_films[nuf_idx[2]]: 5.0,
    new_user_films[nuf_idx[3]]: 3.0,
    new_user_films[nuf_idx[4]]: 4.0,
    new_user_films[nuf_idx[5]]: 4.5,
    new_user_films[nuf_idx[6]]: 5.0
}

In [43]:
new_user_ratings

{'I Still Know What You Did Last Summer (1998)': 5.0,
 'Wedding Singer, The (1998)': 4.5,
 'Donnie Darko (2001)': 5.0,
 'Fear and Loathing in Las Vegas (1998)': 3.0,
 'King of Kong, The (2007)': 4.0,
 'Little Voice (1998)': 4.5,
 'From Russia with Love (1963)': 5.0}

In [44]:
movies_with_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [45]:
movies_with_ratings_new = movies_with_ratings.copy()
movies_with_ratings_new.drop(columns=['timestamp', 'genres'], inplace=True)
movies_with_ratings_new.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


Мы подготовили датасет со всеми оценками всех пользователей, выделим из него фильмы, вошедшие в топ-1000:

In [46]:
top_films_titles = [film[0] for film in top_films]
movies_with_ratings_new_top = movies_with_ratings_new[movies_with_ratings_new.title.isin(top_films_titles)]
movies_with_ratings_new_top.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [47]:
movies_with_ratings_new_top.shape

(61148, 4)

In [48]:
movies_with_ratings_new_top.isnull().any().any()

False

Изменим немного датафрейм:

In [49]:
df = pd.DataFrame({
    'uid': movies_with_ratings_new_top.userId,
    'iid': movies_with_ratings_new_top.title,
    'rating': movies_with_ratings_new_top.rating
})
df.tail(3)

,uid,iid,rating
85133,603,All About Eve (1950),5.0
85134,606,All About Eve (1950),4.0
99604,509,Emma (1996),3.5


**Вот теперь магия гибридного подхода: добавляем данные нового пользователя в наш датафрейм.**

In [50]:
print(np.max(ratings.userId))
print(ratings.userId.nunique())
new_user_id = np.max(ratings.userId) + 1
print(new_user_id)

for film, rating in new_user_ratings.items():
    try:
        df = df.append({
            'uid': new_user_id,
            'iid': film,
            'rating': rating
        }, ignore_index=True)
    except:
        continue

df.tail(10)

610
610
611


,uid,iid,rating
61145,603,All About Eve (1950),5.0
61146,606,All About Eve (1950),4.0
61147,509,Emma (1996),3.5
61148,611,I Still Know What You Did Last Summer (1998),5.0
61149,611,"Wedding Singer, The (1998)",4.5
61150,611,Donnie Darko (2001),5.0
61151,611,Fear and Loathing in Las Vegas (1998),3.0
61152,611,"King of Kong, The (2007)",4.0
61153,611,Little Voice (1998),4.5
61154,611,From Russia with Love (1963),5.0


In [51]:
df.shape

(61155, 3)

---

### Применим алгоритм [SVDpp](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp)

In [52]:
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [53]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))

In [54]:
data = Dataset.load_from_df(df, reader)

In [55]:
%%time
algo = SVDpp()
trainset = data.build_full_trainset()
algo.fit(trainset)

Wall time: 3min 20s


---

### Список рекомендаций для нового пользователя.

In [56]:
#Зададим число рекомендованных новому пользователю фильмов
Number_rs_films = 25

Зная предпочтения нового пользователя можем оценить и соответственно предложить новому пользователю 25 лучших фильмов из топ-1000 (оценим, какой рейтинг он бы поставил фильмам).

In [57]:
new_user_predicted_ratings = {}
for movie in tqdm_notebook(top_films_titles):
    if movie in new_user_ratings:
        continue
    new_user_predicted_ratings[movie] = algo.predict(uid=new_user_id, iid=movie)

Отсортитруем полученные результаты:

In [58]:
recommendations = sorted(new_user_predicted_ratings.items(), key=lambda x: x[1].est, reverse=True)[:Number_rs_films]


**Выведем результаты:**

In [60]:
print(f'{len(recommendations)} лучших фильмов для пользователя с ID={new_user_id}:\n')
print(f"{'#': <4} {'Название': <90} Ожидаемая оценка")
for index, r in enumerate(recommendations):
    print(f'{index + 1: <4} {r[1].iid: <90} {round(r[1].est, 2)}')

25 лучших фильмов для пользователя с ID=611:

#    Название                                                                                   Ожидаемая оценка
1    Fight Club (1999)                                                                          5.0
2    Memento (2000)                                                                             4.98
3    Snatch (2000)                                                                              4.88
4    Lord of the Rings: The Fellowship of the Ring, The (2001)                                  4.87
5    Departed, The (2006)                                                                       4.86
6    Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)                                       4.83
7    Godfather: Part II, The (1974)                                                             4.83
8    Requiem for a Dream (2000)                                                                 4.82
9    WALL·E (2008)                